In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from implementations import *

In [3]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

We have the only one column with integer values (it looks like clusters), so we divide the dataset by this value

In [4]:
np.unique(tX[:,22])

array([0., 1., 2., 3.])

In [5]:
tX0=np.copy(tX[tX[:,22]==0,:])
ids0=np.copy(ids[tX[:,22]==0])
y0 = np.copy(y[tX[:,22]==0])

tX1=np.copy(tX[tX[:,22]==1,:])
ids1=np.copy(ids[tX[:,22]==1])
y1 = np.copy(y[tX[:,22]==1])

tX2=np.copy(tX[tX[:,22]==2,:])
ids2=np.copy(ids[tX[:,22]==2])
y2 =  np.copy(y[tX[:,22]==2])

tX3=np.copy(tX[tX[:,22]==3,:])
ids3=np.copy(ids[tX[:,22]==3])
y3 =  np.copy(y[tX[:,22]==3])

print('Shapes of clusters are: ',tX0.shape,tX1.shape,tX2.shape,tX3.shape)

Shapes of clusters are:  (99913, 30) (77544, 30) (50379, 30) (22164, 30)


In [6]:
#Deleting columns with clusters
tX0=np.copy(np.delete(tX0,(22),axis=1))
tX1=np.copy(np.delete(tX1,(22),axis=1))
tX2=np.copy(np.delete(tX2,(22),axis=1))
tX3=np.copy(np.delete(tX3,(22),axis=1))

In [7]:
#Counting NaNs in columns in each cluster
print('Amount of NaNs in 0 cluster: \n',np.count_nonzero(tX0==-999.0, axis = 0))
print('Amount of NaNs in 1 cluster: \n',np.count_nonzero(tX1==-999.0, axis = 0))
print('Amount of NaNs in 2 cluster: \n',np.count_nonzero(tX2==-999.0, axis = 0))
print('Amount of NaNs in 3 cluster: \n',np.count_nonzero(tX3==-999.0, axis = 0))

Amount of NaNs in 0 cluster: 
 [26123     0     0     0 99913 99913 99913     0     0     0     0     0
 99913     0     0     0     0     0     0     0     0     0 99913 99913
 99913 99913 99913 99913     0]
Amount of NaNs in 1 cluster: 
 [ 7562     0     0     0 77544 77544 77544     0     0     0     0     0
 77544     0     0     0     0     0     0     0     0     0     0     0
     0 77544 77544 77544     0]
Amount of NaNs in 2 cluster: 
 [2952    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0]
Amount of NaNs in 3 cluster: 
 [1477    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0]


We see that in clusters 0 and 1 some columns consist only of NaNs That's why we will delete these columns.

In [8]:
#Deleting columns where all values are same
tX0cl=np.copy(tX0[:,np.invert(np.all(tX0 == tX0[0,:], axis = 0))])
tX1cl=np.copy(tX1[:,np.invert(np.all(tX1 == tX1[0,:], axis = 0))])
tX2cl=np.copy(tX2[:,np.invert(np.all(tX2 == tX2[0,:], axis = 0))])
tX3cl=np.copy(tX3[:,np.invert(np.all(tX3 == tX3[0,:], axis = 0))])

Now we have cleaned all columns except the first one. We have different ways to work with it

#### 1.Delete rows with NaNs in the first column

In [9]:
#Don't overwrite global variables
tX0=np.copy(tX0cl)
tX1=np.copy(tX1cl)
tX2=np.copy(tX2cl)
tX3=np.copy(tX3cl)

In [10]:
#Deleting rows with NaNs
ids0d=np.copy(ids0[tX0[:,0]!=-999.0])
y0d = np.copy(y0[tX0[:,0]!=-999.0])
tX0=np.copy(tX0[tX0[:,0]!=-999.0,:])

ids1d=np.copy(ids1[tX1[:,0]!=-999.0])
y1d = np.copy(y1[tX1[:,0]!=-999.0])
tX1=np.copy(tX1[tX1[:,0]!=-999.0,:])

ids2d=np.copy(ids2[tX2[:,0]!=-999.0])
y2d = np.copy(y2[tX2[:,0]!=-999.0])
tX2=np.copy(tX2[tX2[:,0]!=-999.0,:])

ids3d=np.copy(ids3[tX3[:,0]!=-999.0])
y3d = np.copy(y3[tX3[:,0]!=-999.0])
tX3=np.copy(tX3[tX3[:,0]!=-999.0,:])

##### 1.1. See the correlation between regressors

Now let's see correlation between regressors

In [11]:
#Correlation between regressors in tX0
for i in range(len(tX0[0,:])):
    for j in range(i+1,len(tX0[0,:])):
        corr = np.corrcoef(tX0[:,i],tX0[:,j])
        if np.abs(np.corrcoef(tX0[:,i],tX0[:,j])[0,1])> 0.7:
            print("Pearson correlation between column {i} and {j} : {corr}".format(i=i,j=j,corr=corr[0,1]))

Pearson correlation between column 0 and 2 : 0.9428649994377921
Pearson correlation between column 1 and 7 : 0.7025329814432556
Pearson correlation between column 3 and 5 : 0.9999999999991431
Pearson correlation between column 6 and 9 : 0.7921536842510372
Pearson correlation between column 6 and 12 : 0.7801737531532945


So it makes sense to drop columns 0,3,6

In [12]:
#Dropping columns
tX0=tX0[:,[i for i in range(tX0.shape[1]) if i not in [0,3,6]]]

In [13]:
#Correlation between regressors in tX1
for i in range(len(tX1[0,:])):
    for j in range(i+1,len(tX1[0,:])):
        corr = np.corrcoef(tX1[:,i],tX1[:,j])
        if np.abs(np.corrcoef(tX1[:,i],tX1[:,j])[0,1])> 0.7:
            print("Pearson correlation between column {i} and {j} : {corr}".format(i=i,j=j,corr=corr[0,1]))

Pearson correlation between column 0 and 2 : 0.9203955805006697
Pearson correlation between column 3 and 6 : 0.880515673723393
Pearson correlation between column 3 and 15 : 0.7067535021056447
Pearson correlation between column 3 and 17 : 0.719783491337637
Pearson correlation between column 3 and 18 : 0.9473952774313399
Pearson correlation between column 3 and 21 : 0.9473952764775125
Pearson correlation between column 6 and 17 : 0.7682569234879949
Pearson correlation between column 6 and 18 : 0.9128480282626462
Pearson correlation between column 6 and 21 : 0.9128480302418414
Pearson correlation between column 7 and 12 : 0.7106467382693462
Pearson correlation between column 17 and 18 : 0.721078263561643
Pearson correlation between column 17 and 21 : 0.7210782626271482
Pearson correlation between column 18 and 21 : 0.9999999999990334


Let's drop columns 0,3,6,21

In [14]:
#Dropping columns
tX1=tX1[:,[i for i in range(tX1.shape[1]) if i not in [0,3,6,21]]]

In [15]:
#Correlation between regressors in tX2
for i in range(len(tX2[0,:])):
    for j in range(i+1,len(tX2[0,:])):
        corr = np.corrcoef(tX2[:,i],tX2[:,j])
        if np.abs(np.corrcoef(tX2[:,i],tX2[:,j])[0,1])> 0.7:
            print("Pearson correlation between column {i} and {j} : {corr}".format(i=i,j=j,corr=corr[0,1]))

Pearson correlation between column 0 and 2 : 0.8882697562122746
Pearson correlation between column 3 and 9 : 0.8100583553617612
Pearson correlation between column 3 and 19 : 0.7727347641970795
Pearson correlation between column 3 and 21 : 0.7106329242566304
Pearson correlation between column 3 and 22 : 0.8064082142710389
Pearson correlation between column 3 and 28 : 0.7651054026528024
Pearson correlation between column 4 and 5 : 0.8102770843828327
Pearson correlation between column 4 and 6 : -0.8533511261674986
Pearson correlation between column 5 and 6 : -0.7806386730593765
Pearson correlation between column 9 and 21 : 0.8630680375230015
Pearson correlation between column 9 and 22 : 0.9212004546370948
Pearson correlation between column 9 and 28 : 0.9417446400854351
Pearson correlation between column 10 and 16 : 0.7427175522749608
Pearson correlation between column 21 and 22 : 0.8087380200255875
Pearson correlation between column 21 and 28 : 0.8269810542575168
Pearson correlation betwe

Drop columns 0,3,4,9,22

In [16]:
#Dropping columns
tX2=tX2[:,[i for i in range(tX2.shape[1]) if i not in [0,3,4,9,22]]]

In [17]:
#Correlation between regressors in tX3
for i in range(len(tX3[0,:])):
    for j in range(i+1,len(tX3[0,:])):
        corr = np.corrcoef(tX3[:,i],tX3[:,j])
        if np.abs(np.corrcoef(tX3[:,i],tX3[:,j])[0,1])> 0.7:
            print("Pearson correlation between column {i} and {j} : {corr}".format(i=i,j=j,corr=corr[0,1]))

Pearson correlation between column 0 and 2 : 0.8939715045042343
Pearson correlation between column 3 and 19 : 0.7852631621337898
Pearson correlation between column 4 and 5 : 0.758652958037284
Pearson correlation between column 4 and 6 : -0.7768024192719148
Pearson correlation between column 9 and 21 : 0.9209360084498386
Pearson correlation between column 9 and 22 : 0.8823253774045843
Pearson correlation between column 9 and 25 : 0.7709015614616341
Pearson correlation between column 9 and 28 : 0.957878606826293
Pearson correlation between column 10 and 16 : 0.7636779915431535
Pearson correlation between column 21 and 22 : 0.817603943920867
Pearson correlation between column 21 and 25 : 0.7259662973246287
Pearson correlation between column 21 and 28 : 0.8968125698294412
Pearson correlation between column 22 and 28 : 0.8874163028182186
Pearson correlation between column 25 and 28 : 0.83669471775916


Drop columns 0,9,21,22

In [18]:
#Dropping columns
tX3=tX3[:,[i for i in range(tX3.shape[1]) if i not in [0,9,21,22]]]

In [19]:
#TODO Check correlation between regressors and the result value

In [20]:
#Data standartization
tX0_11=standartize(tX0)
tX1_11=standartize(tX1)
tX2_11=standartize(tX2)
tX3_11=standartize(tX3)

### Models creation (tX0_11..tX3_11 to y0d...y3d)

###### Linear regression

In [21]:
y = [y0d, y1d, y2d, y3d]
tXst = [tX0_11, tX1_11, tX2_11, tX3_11]
def LinearRegressionSubmission(y, tXst):
    w_list = []
    rmse_list = []
    for i in range(len(y)):
        w, mse = least_squares(y[i], tXst[i])
        w_list.append(w)
        rmse_list.append(np.sqrt(2*mse))
    return w_list, rmse_list
LinearRegressionSubmission(y, tXst)

([array([-2.06880312e-01, -4.58597236e-01,  3.52838215e-01,  3.23421071e-02,
         -1.58959755e-01, -1.15919349e-02,  2.81667771e-01,  9.47439000e-05,
          4.26477717e-04,  1.65336871e-01,  4.68799168e-03,  6.38468391e-04,
         -2.37790473e-02, -6.95609668e-03,  3.95805980e-02]),
  array([-2.06301728e-01, -2.50211556e-01,  2.76199667e-01,  3.96837306e-02,
         -1.91784626e-01,  1.18899478e-01,  1.77286403e-01,  3.66955178e-03,
         -6.06497299e-03,  2.48889753e-01, -8.70445912e-03, -8.68657328e-07,
          4.95246061e-02,  4.34505291e-03, -2.81561347e-02,  5.48007959e-02,
         -5.30974371e-04, -2.24981539e-03]),
  array([-0.16371398, -0.15131667,  0.20788056,  0.00735713,  0.22116094,
         -0.0555752 , -0.15740836,  0.13985692,  0.18505956,  0.21305616,
         -0.00308169,  0.00068213,  0.2712701 , -0.00727277,  0.00293023,
          0.15992448,  0.0052238 , -0.11861323,  0.00838199,  0.00867409,
          0.0940448 ,  0.0048366 , -0.00577548, -0.1505115

###### Linear regression with polynomial expansion

In [22]:
def polynomial_regression(y,tXst):
    """Constructing the polynomial basis function expansion of the data,
       and then running least squares regression."""
    # define parameters
    degrees = [i for i in range(12)]
    mean_rmse = np.zeros((4,len(degrees)))
    k_fold = 4
    seed = 1
    rmse_tr = []
    rmse_te = []
    test = []
    deg = []
    minimum = []
    for ind,degree in enumerate(degrees):
        for i in range(0,4):
            print("Subset {i}, degree {degree}".format(i = i, degree=degree))
            k_indices,indices = build_k_indices(y[i], k_fold,seed)
            loss_tr=0
            loss_te=0
            rmse_te_tmp = []
            for k in range(k_fold):
                l_tr, l_te = cross_validation_leastsquares(y[i], tXst[i], k_indices, k,degree)
                #loss_tr+=l_tr
                loss_te+=l_te
                #rmse_tr_tmp.append(l_te)
            mean_rmse[i][ind]=loss_te/k_fold
            print(mean_rmse[i][ind])
            
    for i in range(0,4):
        test.append(mean_rmse[i].tolist())
        #print(test[i])
        deg.append(degrees[np.argmin(mean_rmse[i])])
        minimum.append(min(test[i]))
        print("Best degree for subset {i}: {degree}, test error: {err}".format(i = i,degree=degrees[np.argmin(mean_rmse[i])],err=min(test[i])))
    return deg , minimum

In [23]:
polynomial_regression(y,tXst)

Subset 0, degree 0
0.9362851108032995
Subset 1, degree 0
0.9736332394593022
Subset 2, degree 0
0.9978568407102907
Subset 3, degree 0
0.9331902231837701
Subset 0, degree 1
0.7919284693081986
Subset 1, degree 1
0.8859956769469683
Subset 2, degree 1
0.851156681153518
Subset 3, degree 1
0.86537897664666
Subset 0, degree 2
2.564567067352238
Subset 1, degree 2
0.8446145338160909
Subset 2, degree 2
0.81718062401826
Subset 3, degree 2
0.8306381201178299
Subset 0, degree 3
81.63576812355517
Subset 1, degree 3
0.8421616433385937
Subset 2, degree 3
0.8036165091579167
Subset 3, degree 3
0.8199369424492217
Subset 0, degree 4
549.5368043380818
Subset 1, degree 4
0.8510783592721152
Subset 2, degree 4
0.8441901470491319
Subset 3, degree 4
0.8198653741363071
Subset 0, degree 5
804739.7968592497
Subset 1, degree 5
0.8745202202262206
Subset 2, degree 5
1.2959233522768474
Subset 3, degree 5
1.1069260678625246
Subset 0, degree 6
45383418.163722746
Subset 1, degree 6
1.0389525450967274
Subset 2, degree 6
2.

([1, 3, 3, 4],
 [0.7919284693081986,
  0.8421616433385937,
  0.8036165091579167,
  0.8198653741363071])

###### Gradient descent

In [52]:
def GradientDescentSubmission(y, tXst):
  # Define the parameters of the algorithm.
    max_iters = 1000
    gamma = 0.5
  # Initialization
    
    w_list = []
    loss_list = []
  # Start GD
    for i in range(4):
        w_initial = np.array([np.random.rand() for i in range(tXst[i].shape[1])])
        gradient_loss, gradient_w = gradient_descent(y[i], tXst[i], w_initial, max_iters, gamma)
        w_list.append(gradient_w)
        loss_list.append(gradient_loss)
    return w_list, loss_list
GradientDescentSubmission(y,tXst)

([array([-2.06880312e-01, -4.58597236e-01,  3.52838215e-01,  3.23421071e-02,
         -1.58959755e-01, -1.15919349e-02,  2.81667771e-01,  9.47439000e-05,
          4.26477717e-04,  1.65336871e-01,  4.68799168e-03,  6.38468391e-04,
         -2.37790474e-02, -6.95609668e-03,  3.95805980e-02]),
  array([-2.06301728e-01, -2.50211556e-01,  2.76199667e-01,  3.96837306e-02,
         -1.91784626e-01,  1.18899478e-01,  1.77286403e-01,  3.66955178e-03,
         -6.06497299e-03,  2.48889753e-01, -8.70445912e-03, -8.68657328e-07,
          4.95246061e-02,  4.34505291e-03, -2.81561347e-02,  5.48007959e-02,
         -5.30974371e-04, -2.24981539e-03]),
  array([-0.16371398, -0.15131667,  0.20788056,  0.00735713,  0.22116094,
         -0.0555752 , -0.15740836,  0.13985692,  0.18505956,  0.21305616,
         -0.00308169,  0.00068213,  0.2712701 , -0.00727277,  0.00293023,
          0.15992448,  0.0052238 , -0.11861323,  0.00838199,  0.00867409,
          0.0940448 ,  0.0048366 , -0.00577548, -0.1505115

In [90]:
np.random.rand()

0.7626855144634183

###### Stochastic gradient descent

In [24]:
def StochasticGradientDescent(y, tXst):
  # Define the parameters of the algorithm.
    max_iters = 100
    gamma = 0.001
    batch_size = 1
  # Initialization
    w_list = []
    loss_list = []
  # Start SGD.
    for i in range(4):
        w_initial = np.array([0 for i in range(tXst[i].shape[1])])
        sgd_losses, sgd_w = stochastic_gradient_descent(y[i], tXst[i], w_initial, batch_size, max_iters, gamma)
        w_list.append(sgd_w)
        loss_list.append(sgd_losses)
    return w_list, loss_list
StochasticGradientDescent(y, tXst)

([array([-0.03953648, -0.00835486,  0.01807197, -0.00663377, -0.02433187,
          0.01337897,  0.01250804,  0.00348614, -0.00726161, -0.01795264,
         -0.0077468 ,  0.01124808, -0.02361948, -0.01893504,  0.00024943]),
  array([-0.02351591,  0.00108344,  0.01301907, -0.00146986, -0.03370089,
          0.02917919,  0.02215244, -0.00269818, -0.01068807, -0.01748665,
         -0.00024932, -0.00090134,  0.00666417, -0.00540354,  0.00534665,
          0.01400693, -0.00442097,  0.0080783 ]),
  array([-0.01726204, -0.00730088,  0.03067101, -0.0321546 , -0.00369207,
         -0.00240034, -0.02041713,  0.00798676,  0.03914496,  0.02928135,
          0.00298133, -0.00496922, -0.00728747, -0.00343598,  0.01795668,
         -0.00532421, -0.00087185, -0.00793809, -0.00393357, -0.02118535,
         -0.00500613,  0.0070464 ,  0.00690569, -0.00915552]),
  array([-0.01108834,  0.00796019,  0.02183295, -0.00056793, -0.00041752,
         -0.00297566, -0.00638126,  0.00488044, -0.00836563,  0.0225326

###### Ridge regression

In [25]:
def cross_validation_demo():
    seed = 1
    k_fold = 4
    lambdas = np.logspace(-2, 1, 8)
    # split data in k fold
    num_degrees=3
    # define lists to store the loss of training data and test data
    matrix_te0=np.zeros(shape=(num_degrees,len(lambdas)))
    matrix_tr0=np.zeros(shape=(num_degrees,len(lambdas)))
    matrix_te1=np.zeros(shape=(num_degrees,len(lambdas)))
    matrix_tr1=np.zeros(shape=(num_degrees,len(lambdas)))
    matrix_te2=np.zeros(shape=(num_degrees,len(lambdas)))
    matrix_tr2=np.zeros(shape=(num_degrees,len(lambdas)))
    matrix_te3=np.zeros(shape=(num_degrees,len(lambdas)))
    matrix_tr3=np.zeros(shape=(num_degrees,len(lambdas)))
    for degree in np.arange(num_degrees):
        for ind_lmbd,lambda_ in enumerate(lambdas):
            #losses_te=[]
            #losses_tr=[]
            for i in range(0,4):
                k_indices,indices = build_k_indices(y[i], k_fold, seed)
                loss_tr=0
                loss_te=0
                for k in range(k_fold):
                    #print([degree,ind_lmbd,i,k])
                    l_tr, l_te,_ = cross_validation_ridge(y[i], tXst[i], k_indices, k, lambda_, degree)
                    loss_tr+=l_tr
                    loss_te+=l_te
                if i==0:
                    matrix_te0[degree][ind_lmbd]=loss_te/k_fold
                    matrix_tr0[degree][ind_lmbd]=loss_tr/k_fold
                elif i==1:
                    matrix_te1[degree][ind_lmbd]=loss_te/k_fold
                    matrix_tr1[degree][ind_lmbd]=loss_tr/k_fold
                elif i==2:
                    matrix_te2[degree][ind_lmbd]=loss_te/k_fold
                    matrix_tr2[degree][ind_lmbd]=loss_tr/k_fold
                elif i==3:
                    matrix_te3[degree][ind_lmbd]=loss_te/k_fold
                    matrix_tr3[degree][ind_lmbd]=loss_tr/k_fold
                #losses_te.append(np.mean(loss_te))
                #losses_tr.append(np.mean(loss_tr))
            #matrix_tr[degree][ind_lmbd]= losses_tr
            #matrix_te[degree][ind_lmbd]=losses_te
    
   
    
    #get the best degree lambda couple  
    result = np.where(matrix_te0 == np.amin(matrix_te0))
    listOfCoordinates = list(zip(result[0], result[1]))
    best_degree = listOfCoordinates[0][0]
    best_lambda = listOfCoordinates[0][1]
    print('Best parameters for 0 subset are:',best_degree,lambdas[best_lambda],' with error: ',matrix_te0[best_degree][best_lambda])
    
    result = np.where(matrix_te1 == np.amin(matrix_te1))
    listOfCoordinates = list(zip(result[0], result[1]))
    best_degree = listOfCoordinates[0][0]
    best_lambda = listOfCoordinates[0][1]
    print('Best parameters for 1 subset are:',best_degree,lambdas[best_lambda],' with error: ',matrix_te1[best_degree][best_lambda])
    
    result = np.where(matrix_te2 == np.amin(matrix_te2))
    listOfCoordinates = list(zip(result[0], result[1]))
    best_degree = listOfCoordinates[0][0]
    best_lambda = listOfCoordinates[0][1]
    print('Best parameters for 2 subset are:',best_degree,lambdas[best_lambda],' with error: ',matrix_te2[best_degree][best_lambda])
    
    result = np.where(matrix_te3 == np.amin(matrix_te3))
    listOfCoordinates = list(zip(result[0], result[1]))
    best_degree = listOfCoordinates[0][0]
    best_lambda = listOfCoordinates[0][1]
    print('Best parameters for 3 subset are:',best_degree,lambdas[best_lambda],' with error: ',matrix_te3[best_degree][best_lambda])
    
    print(matrix_te0,matrix_te1,matrix_te2,matrix_te3)
    #show cross validation for each degree

cross_validation_demo()

Best parameters for 0 subset are: 1 0.01  with error:  0.7929677144488401
Best parameters for 1 subset are: 2 0.01  with error:  0.8450561850194014
Best parameters for 2 subset are: 2 0.01  with error:  0.8173424201564052
Best parameters for 3 subset are: 2 0.01  with error:  0.8320016918872625
[[0.93630976 0.93645421 0.93732362 0.94137645 0.95317962 0.97125258
  0.98617216 0.99424679]
 [0.79296771 0.7959668  0.80461279 0.82188364 0.85171029 0.89660549
  0.94290872 0.97407404]
 [2.63784812 2.50596282 2.11248506 1.4968839  0.92503176 0.94594117
  1.17291633 1.16651706]] [[0.97364297 0.97370116 0.9740531  0.97569909 0.98052021 0.98797365
  0.99418946 0.99757674]
 [0.88634788 0.88770695 0.89194932 0.90119727 0.91843812 0.94433812
  0.96988659 0.98647535]
 [0.84505619 0.84631685 0.84932718 0.85628768 0.8726649  0.90067554
  0.93105463 0.95601466]] [[0.99785752 0.99786204 0.99788998 0.9980218  0.99841024 0.99901492
  0.99952264 0.9998006 ]
 [0.85156878 0.85317203 0.85792353 0.86728612 0.883

###### Logistic regression

In [23]:
def sigmoid(x):
    t=np.copy(x)
    t[t>0]=1/(1+np.exp(-t[t>0]))
    t[t<0]=np.exp(t[t<0])/(1+np.exp(t[t<0]))
    return t

def calculate_loss(y, tx, w):
    pred = sigmoid(tx.dot(w))
    loss=((y+1)/2.0).T.dot(np.log(pred+1e-20)) + (1 - ((y+1)/2.0)).T.dot(np.log(1 - pred+1e-20))
    return np.squeeze(loss)

def calculate_gradient(y, tx, w):
    pred = sigmoid(tx.dot(w))
    grad = tx.T.dot(pred-(y+1)/2.0)
    return grad

def learning_by_gradient_descent(y, tx, w, gamma):
    loss=calculate_loss(y,tx,w)
    grad = calculate_gradient(y,tx,w)
    w = w-gamma*grad
    return loss, w

In [24]:
def logistic_regression_gradient_descent_demo(y, x):
    # init parameters
    max_iter = 10000
    threshold = 1e-6
    gamma = 0.00001
    all_losses=[]
    weights=[]
    
    for i in range(0,4):
        losses = []
        
        # build tx and ty
        tx = np.c_[np.ones((y[i].shape[0], 1)), x[i]]
        w = np.random.rand(tx.shape[1], 1)
        ty=y[i].reshape(y[i].shape[0],1)

        print('Training for subset {0}\n'.format(i))
        # start the logistic regression
        for iter in range(max_iter):
            
            # get loss and update w
            loss, w = learning_by_gradient_descent(ty, tx, w, gamma)
            
            # log info
            if iter % 100 == 0:
                print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
            
            #store loss
            losses.append(loss)
            
            # converge criterion
            if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
                break

        print("\nBest loss for subset {i}={l}\n".format(i=i,l=calculate_loss(ty, tx, w)))
        all_losses.append(losses)
        weights.append(w)
    return all_losses,weights

In [45]:
loss,w=logistic_regression_gradient_descent_demo(y,tXst)

Training for subset 0

Current iteration=0, loss=-94886.20696476189
Current iteration=100, loss=-34074.19555855833
Current iteration=200, loss=-33528.1465416088
Current iteration=300, loss=-33359.41626291181
Current iteration=400, loss=-33297.208698572635
Current iteration=500, loss=-33272.00601237887
Current iteration=600, loss=-33261.18326195566
Current iteration=700, loss=-33256.34954841683
Current iteration=800, loss=-33254.1290540888
Current iteration=900, loss=-33253.08727079505
Current iteration=1000, loss=-33252.59046749104
Current iteration=1100, loss=-33252.35048365808
Current iteration=1200, loss=-33252.23335691195
Current iteration=1300, loss=-33252.17571409414
Current iteration=1400, loss=-33252.14715351522
Current iteration=1500, loss=-33252.13292457066
Current iteration=1600, loss=-33252.125803966315
Current iteration=1700, loss=-33252.12222764617
Current iteration=1800, loss=-33252.120426144036
Current iteration=1900, loss=-33252.119516503204
Current iteration=2000, los

In [197]:
#Accuracy
for i in range(4):
    print('Accuracy for subset {i} is {acc}'.format(i=i,acc=np.sum(np.where(sigmoid(np.c_[np.ones((y[i].shape[0], 1)), tXst[i]]@w[i])>0.5,1,0)==((y[i].reshape(y[i].shape[0],1)+1)/2.0).astype('int'))/y[i].shape[0]))

Accuracy for subset 0 is 0.7878845372001626
Accuracy for subset 1 is 0.6924494870109457
Accuracy for subset 2 is 0.7306175807029751
Accuracy for subset 3 is 0.7179871416831827


###### Penalized logistic regression

In [25]:
def penalized_logistic_regression(y, tx, w, lambda_):
    """return the loss, gradient"""
    
    loss = calculate_loss(y, tx, w) + lambda_ * (w.T.dot(w))
    gradient = calculate_gradient(y, tx, w) + 2 * lambda_ * w
    return loss, gradient

def learning_by_penalized_gradient(y, tx, w, gamma, lambda_):
    loss,gradient = penalized_logistic_regression(y,tx,w,lambda_)
    w = w-gradient*gamma
    return loss, w

In [32]:
def logistic_regression_penalized_gradient_descent_demo(y, x, max_iters = 10000,gamma = 0.0001,lambda_ = 0.01):
    # init parameters
    threshold = 1e-6
    all_losses=[]
    weights=[]
    for i in range(0,len(y)):
        losses = []
    
       # build tx and ty\n",
        tx = np.c_[np.ones((y[i].shape[0], 1)), x[i]]
        w = np.random.rand(tx.shape[1], 1)
        ty=y[i].reshape(y[i].shape[0],1)
        print('Training for subset {0}\n'.format(i))
        # start the logistic regression
        for iter in range(max_iters):
    
             # get loss and update w.
            loss, w = learning_by_penalized_gradient(ty, tx, w, gamma, lambda_)\
           # log info
            if iter % 100 == 0:
                print("Current iteration={i}, loss={l}".format(i=iter, l=np.squeeze(loss)))

                losses.append(loss)
                if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
                    break

        print("\nBest loss for subset {i}={l}\n".format(i=i,l=calculate_loss(ty, tx, w)))
        all_losses.append(losses)
        weights.append(w)
    return all_losses,weights

In [33]:
logistic_regression_penalized_gradient_descent_demo(y, tXst)

Training for subset 0

Current iteration=0, loss=-87086.59742854706
Current iteration=100, loss=-38262.36958564681
Current iteration=200, loss=-38228.53855158689
Current iteration=300, loss=-38227.83331259681
Current iteration=400, loss=-38227.81061658162
Current iteration=500, loss=-38227.80964725577
Current iteration=600, loss=-38227.80960101806
Current iteration=700, loss=-38227.80959873831
Current iteration=800, loss=-38227.80959862493
Current iteration=900, loss=-38227.80959861927

Best loss for subset 0=-41634.19379877557

Training for subset 1

Current iteration=0, loss=-86437.1472959503
Current iteration=100, loss=-52368.28844696923
Current iteration=200, loss=-52348.067606342796
Current iteration=300, loss=-52347.9374656172
Current iteration=400, loss=-52347.936627008705
Current iteration=500, loss=-52347.936621604735
Current iteration=600, loss=-52347.936621569934
Current iteration=700, loss=-52347.936621569715

Best loss for subset 1=-61146.758356030376

Training for subset 

([[array([[-87086.59742855]]),
   array([[-38262.36958565]]),
   array([[-38228.53855159]]),
   array([[-38227.8333126]]),
   array([[-38227.81061658]]),
   array([[-38227.80964726]]),
   array([[-38227.80960102]]),
   array([[-38227.80959874]]),
   array([[-38227.80959862]]),
   array([[-38227.80959862]])],
  [array([[-86437.14729595]]),
   array([[-52368.28844697]]),
   array([[-52348.06760634]]),
   array([[-52347.93746562]]),
   array([[-52347.93662701]]),
   array([[-52347.9366216]]),
   array([[-52347.93662157]]),
   array([[-52347.93662157]])],
  [array([[-50896.67791622]]),
   array([[-31151.91216584]]),
   array([[-31140.92274757]]),
   array([[-31140.87009055]]),
   array([[-31140.86981938]]),
   array([[-31140.86981792]]),
   array([[-31140.86981791]])],
  [array([[-35375.29452632]]),
   array([[-11288.86901108]]),
   array([[-11283.04050497]]),
   array([[-11282.49287681]]),
   array([[-11282.43454064]]),
   array([[-11282.42809782]]),
   array([[-11282.42733804]]),
   arra

In [36]:
def cross_validation_logistic_regression(y, x, k_indices,max_iter, k, gamma,lambda_):
    x_test = x[k_indices[k]]
    y_test = y[k_indices[k]]
    x_train = []
    y_train = []

    tr_indice = k_indices[~(np.arange(k_indices.shape[0]) == k)]
    tr_indice = tr_indice.reshape(-1)
    x_train = x[tr_indice]
    y_train = y[tr_indice]
    threshold = 1e-6
    
    w = np.random.rand(x.shape[1], 1)
    # get loss and update w.
    losses=[]
    for iter in range(max_iter):
        l_tr,w = learning_by_penalized_gradient(y_train, x_train, w, gamma, lambda_)
        # log info
        losses.append(l_tr)
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=np.squeeze(l_tr)))
        # converge criterion
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break

    loss_tr = losses[-1]
    loss_te= calculate_loss(y_test, x_test, w)

    return loss_tr, loss_te

In [39]:
def logistic_regression_penalized_gradient_descent_demo(y, x):
    # init parameters
    max_iter = 2000
    gammas = [0.0001]
    lambdas = [0.01]

    all_losses=[]
    weights=[]
    k_fold = 4
    seed = 1
    matrix_te0=np.zeros(shape=(len(gammas),len(lambdas)))
    matrix_te1=np.zeros(shape=(len(gammas),len(lambdas)))
    matrix_te2=np.zeros(shape=(len(gammas),len(lambdas)))
    matrix_te3=np.zeros(shape=(len(gammas),len(lambdas)))

    for i in range(0,len(y)):
        k_indices,indices = build_k_indices(y[i], k_fold, seed)

        # build tx and ty
        tx = np.c_[np.ones((y[i].shape[0], 1)), x[i]]
        ty=y[i].reshape(y[i].shape[0],1)
       
        print('Training for subset {0}\n'.format(i))
        # start the logistic regression
        for ind_gm,gamma in enumerate(gammas):
            for ind_lmbd,lambda_ in enumerate(lambdas):
                loss_te=0
                for k in range(k_fold):
                    l_tr,l_te=cross_validation_logistic_regression(ty, tx, k_indices,max_iter, k, gamma,lambda_)
                    loss_te+=l_te
                if i==0:
                    matrix_te0[ind_gm][ind_lmbd]=loss_te
                elif i==1:
                    matrix_te1[ind_gm][ind_lmbd]=loss_te
                elif i==2:
                    matrix_te2[ind_gm][ind_lmbd]=loss_te
                elif i==3:
                    matrix_te3[ind_gm][ind_lmbd]=loss_te

    result = np.where(matrix_te0 == np.amin(matrix_te0))
    listOfCoordinates = list(zip(result[0], result[1]))
    best_gamma = listOfCoordinates[0][0]
    best_lambda = listOfCoordinates[0][1]
    print('Best parameters for 0 subset are:',gammas[best_gamma],lambdas[best_lambda],' with error: ',matrix_te0[best_gamma][best_lambda])
    
    result = np.where(matrix_te1 == np.amin(matrix_te1))
    listOfCoordinates = list(zip(result[0], result[1]))
    best_gamma = listOfCoordinates[0][0]
    best_lambda = listOfCoordinates[0][1]
    print('Best parameters for 1 subset are:',gammas[best_gamma],lambdas[best_lambda],' with error: ',matrix_te1[best_gamma][best_lambda])
    
    result = np.where(matrix_te2 == np.amin(matrix_te2))
    listOfCoordinates = list(zip(result[0], result[1]))
    best_gamma = listOfCoordinates[0][0]
    best_lambda = listOfCoordinates[0][1]
    print('Best parameters for 2 subset are:',gammas[best_gamma],lambdas[best_lambda],' with error: ',matrix_te2[best_gamma][best_lambda])
    
    result = np.where(matrix_te3 == np.amin(matrix_te3))
    listOfCoordinates = list(zip(result[0], result[1]))
    best_gamma = listOfCoordinates[0][0]
    best_lambda = listOfCoordinates[0][1]
    print('Best parameters for 3 subset are:',gammas[best_gamma],lambdas[best_lambda],' with error: ',matrix_te3[best_gamma][best_lambda])

In [40]:
logistic_regression_penalized_gradient_descent_demo(y, tXst)

Training for subset 0

Current iteration=0, loss=-79693.56666204367
Current iteration=100, loss=-24902.681151507677
Current iteration=200, loss=-24898.961706125283
Current iteration=300, loss=-24898.939356634404
Current iteration=0, loss=-85534.50317046499
Current iteration=100, loss=-24965.26431990047
Current iteration=200, loss=-24963.28460202133
Current iteration=300, loss=-24963.274212070268
Current iteration=0, loss=-81298.66836860028
Current iteration=100, loss=-24993.06800695182
Current iteration=200, loss=-24990.580085344995
Current iteration=300, loss=-24990.566619231726
Current iteration=0, loss=-96870.32202113386
Current iteration=100, loss=-24884.111895406
Current iteration=200, loss=-24881.07193429545
Current iteration=300, loss=-24881.05480022465
Training for subset 1

Current iteration=0, loss=-69852.29851089217
Current iteration=100, loss=-32131.119060250454
Current iteration=200, loss=-32125.966774629076
Current iteration=300, loss=-32125.94469168458
Current iteration=

In [71]:
def logistic_regression_penalized_gradient_descent_demo(y, x):
    # init parameters
    max_iter = 10000
    gamma = 0.00001
    lambda_ = 0.1
    threshold = 1e-8
    all_losses=[]
    weights=[]
    
    for i in range(0,4):
        losses = []
        
        # build tx and ty
        tx = np.c_[np.ones((y[i].shape[0], 1)), x[i]]
        w = np.random.rand(tx.shape[1], 1)
        ty=y[i].reshape(y[i].shape[0],1)
        
        print('Training for subset {0}\n'.format(i))
        # start the logistic regression
        for iter in range(max_iter):
            
            # get loss and update w.
            loss, w = learning_by_penalized_gradient(ty, tx, w, gamma, lambda_)
            
            # log info
            if iter % 100 == 0:
                print("Current iteration={i}, loss={l}".format(i=iter, l=np.squeeze(loss)))
            # converge criterion
            losses.append(loss)
            if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
                break
    # visualization
    #visualization(y, x, mean_x, std_x, w, "classification_by_logistic_regression_penalized_gradient_descent",True)
    #print("loss={l}".format(l=calculate_loss(y, tx, w)))
        print("\nBest loss for subset {i}={l}\n".format(i=i,l=calculate_loss(ty, tx, w)))
        all_losses.append(losses)
        weights.append(w)
    return all_losses,weights

In [68]:
loss,w=logistic_regression_penalized_gradient_descent_demo(y, tXst)

Training for subset 0

Current iteration=0, loss=[[-118692.63740843]]
Current iteration=100, loss=[[-33929.39805208]]
Current iteration=200, loss=[[-33492.08202483]]
Current iteration=300, loss=[[-33348.29032852]]
Current iteration=400, loss=[[-33293.08509529]]
Current iteration=500, loss=[[-33269.97014293]]
Current iteration=600, loss=[[-33259.7502091]]
Current iteration=700, loss=[[-33255.05998517]]
Current iteration=800, loss=[[-33252.84781388]]
Current iteration=900, loss=[[-33251.78190969]]
Current iteration=1000, loss=[[-33251.25912803]]
Current iteration=1100, loss=[[-33250.99866268]]
Current iteration=1200, loss=[[-33250.86694308]]
Current iteration=1300, loss=[[-33250.79932134]]
Current iteration=1400, loss=[[-33250.76404525]]
Current iteration=1500, loss=[[-33250.74531453]]
Current iteration=1600, loss=[[-33250.73516964]]
Current iteration=1700, loss=[[-33250.7295514]]
Current iteration=1800, loss=[[-33250.72636285]]
Current iteration=1900, loss=[[-33250.72450517]]
Current it

In [70]:
#Accuracy
w=loss[:]
for i in range(4):
    print('Accuracy for subset {i} is {acc}'.format(i=i,acc=np.sum(np.where(sigmoid(np.c_[np.ones((y[i].shape[0], 1)), tXst[i]]@w[i])>0.5,1,0)==((y[i].reshape(y[i].shape[0],1)+1)/2.0).astype('int'))/y[i].shape[0]))

Accuracy for subset 0 is 0.7878980891719746
Accuracy for subset 1 is 0.6924351976222457
Accuracy for subset 2 is 0.7306386657389251
Accuracy for subset 3 is 0.7180838207569972


##### 1.2. Don't see the correlation between regressors

In [ ]:
#Data standartization
tX0_12=standartize(tX0cl)
tX1_12=standartize(tX1cl)
tX2_12=standartize(tX2cl)
tX3_12=standartize(tX3cl)

### Models creation (tX0_12..tX3_12 to y0d...y3d)

#### 2.Substitute rows with NaNs in the first column by the mean value

In [277]:
#Don't overwrite global variables
tX0=np.copy(tX0cl)
tX1=np.copy(tX1cl)
tX2=np.copy(tX2cl)
tX3=np.copy(tX3cl)

In [278]:
#Calculate mean for 1st column without -999.0 values
mean0=tX0[tX0[:,0]!=-999.0].mean(axis=0)[0]
mean1=tX1[tX1[:,0]!=-999.0].mean(axis=0)[0]
mean2=tX2[tX2[:,0]!=-999.0].mean(axis=0)[0]
mean3=tX3[tX3[:,0]!=-999.0].mean(axis=0)[0]

In [279]:
tX0=np.copy(np.where(tX0==-999.0,mean0,tX0))
tX1=np.copy(np.where(tX1==-999.0,mean1,tX1))
tX2=np.copy(np.where(tX2==-999.0,mean2,tX2))
tX3=np.copy(np.where(tX3==-999.0,mean3,tX3))

##### 2.1. See the correlation between regressors

Now let's see correlation between regressors

In [283]:
#Correlation between regressors in tX0
for i in range(len(tX0[0,:])):
    for j in range(i+1,len(tX0[0,:])):
        corr = np.corrcoef(tX0[:,i],tX0[:,j])
        if np.abs(np.corrcoef(tX0[:,i],tX0[:,j])[0,1])> 0.7:
            print("Pearson correlation between column {i} and {j} : {corr}".format(i=i,j=j,corr=corr[0,1]))

Pearson correlation between column 0 and 2 : 0.7488162009965309
Pearson correlation between column 3 and 5 : 0.9999999999988465
Pearson correlation between column 6 and 9 : 0.8002688533328596
Pearson correlation between column 6 and 12 : 0.7797439568172713


So it makes sense to drop columns 0,3,6

In [284]:
#Dropping columns
tX0=tX0[:,[i for i in range(tX0.shape[1]) if i not in [0,3,6]]]

In [285]:
#Correlation between regressors in tX1
for i in range(len(tX1[0,:])):
    for j in range(i+1,len(tX1[0,:])):
        corr = np.corrcoef(tX1[:,i],tX1[:,j])
        if np.abs(np.corrcoef(tX1[:,i],tX1[:,j])[0,1])> 0.7:
            print("Pearson correlation between column {i} and {j} : {corr}".format(i=i,j=j,corr=corr[0,1]))

Pearson correlation between column 0 and 2 : 0.8445009835185863
Pearson correlation between column 3 and 6 : 0.8632110677076686
Pearson correlation between column 3 and 17 : 0.7125818159025532
Pearson correlation between column 3 and 18 : 0.9367590332144766
Pearson correlation between column 3 and 21 : 0.9367590311733165
Pearson correlation between column 6 and 17 : 0.757990128208487
Pearson correlation between column 6 and 18 : 0.9051662981927852
Pearson correlation between column 6 and 21 : 0.905166300382275
Pearson correlation between column 7 and 12 : 0.709867998636906
Pearson correlation between column 17 and 18 : 0.7068771933564622
Pearson correlation between column 17 and 21 : 0.7068771905369512
Pearson correlation between column 18 and 21 : 0.999999999999017


Let's drop columns 0,3,6,21

In [286]:
#Dropping columns
tX1=tX1[:,[i for i in range(tX1.shape[1]) if i not in [0,3,6,21]]]

In [287]:
#Correlation between regressors in tX2
for i in range(len(tX2[0,:])):
    for j in range(i+1,len(tX2[0,:])):
        corr = np.corrcoef(tX2[:,i],tX2[:,j])
        if np.abs(np.corrcoef(tX2[:,i],tX2[:,j])[0,1])> 0.7:
            print("Pearson correlation between column {i} and {j} : {corr}".format(i=i,j=j,corr=corr[0,1]))

Pearson correlation between column 0 and 2 : 0.8228939521810881
Pearson correlation between column 3 and 9 : 0.800559238241552
Pearson correlation between column 3 and 19 : 0.7285433946032545
Pearson correlation between column 3 and 21 : 0.7069249354905159
Pearson correlation between column 3 and 22 : 0.7995787774292449
Pearson correlation between column 3 and 28 : 0.7584645222360212
Pearson correlation between column 4 and 5 : 0.8098715851425442
Pearson correlation between column 4 and 6 : -0.8501315483608742
Pearson correlation between column 5 and 6 : -0.7780946604694609
Pearson correlation between column 9 and 21 : 0.8523147287795626
Pearson correlation between column 9 and 22 : 0.9191164357409889
Pearson correlation between column 9 and 28 : 0.9397870414625126
Pearson correlation between column 10 and 16 : 0.7438565749365871
Pearson correlation between column 21 and 22 : 0.7970772581576845
Pearson correlation between column 21 and 28 : 0.8147451242277628
Pearson correlation betwee

Drop columns 0,3,4,9,22

In [288]:
#Dropping columns
tX2=tX2[:,[i for i in range(tX2.shape[1]) if i not in [0,3,4,9,22]]]

In [289]:
#Correlation between regressors in tX3
for i in range(len(tX3[0,:])):
    for j in range(i+1,len(tX3[0,:])):
        corr = np.corrcoef(tX3[:,i],tX3[:,j])
        if np.abs(np.corrcoef(tX3[:,i],tX3[:,j])[0,1])> 0.7:
            print("Pearson correlation between column {i} and {j} : {corr}".format(i=i,j=j,corr=corr[0,1]))

Pearson correlation between column 0 and 2 : 0.8172288262177909
Pearson correlation between column 3 and 19 : 0.7502599406025436
Pearson correlation between column 4 and 5 : 0.758966330984028
Pearson correlation between column 4 and 6 : -0.7735086978837818
Pearson correlation between column 9 and 21 : 0.9160946827155181
Pearson correlation between column 9 and 22 : 0.8813251894172176
Pearson correlation between column 9 and 25 : 0.7708057103889608
Pearson correlation between column 9 and 28 : 0.9569498612168339
Pearson correlation between column 10 and 16 : 0.7683893227784884
Pearson correlation between column 21 and 22 : 0.8132272408011297
Pearson correlation between column 21 and 25 : 0.7219329222297065
Pearson correlation between column 21 and 28 : 0.892063808128829
Pearson correlation between column 22 and 28 : 0.887903038587193
Pearson correlation between column 25 and 28 : 0.8365333266992916


Drop columns 0,9,21,22

In [290]:
#Dropping columns
tX3=tX3[:,[i for i in range(tX3.shape[1]) if i not in [0,9,21,22]]]

In [82]:
#TODO Check correlation between regressors and the result value

In [291]:
#Data standartization
tX0_21=standartize(tX0)
tX1_21=standartize(tX1)
tX2_21=standartize(tX2)
tX3_21=standartize(tX3)

### Models creation (tX0_21..tX3_21 to y0...y3)

##### 2.2. Don't the correlation between regressors

In [293]:
#Don't overwrite global variables
tX0=np.copy(tX0cl)
tX1=np.copy(tX1cl)
tX2=np.copy(tX2cl)
tX3=np.copy(tX3cl)

#Replace NaNs with median
tX0=np.copy(np.where(tX0==-999.0,mean0,tX0))
tX1=np.copy(np.where(tX1==-999.0,mean1,tX1))
tX2=np.copy(np.where(tX2==-999.0,mean2,tX2))
tX3=np.copy(np.where(tX3==-999.0,mean3,tX3))

#Data standartization
tX0_22=standartize(tX0)
tX1_22=standartize(tX1)
tX2_22=standartize(tX2)
tX3_22=standartize(tX3)

### Models creation (tX0_22..tX3_22 to y0...y3)

## Generate predictions and save ouput in csv format for submission:

In [10]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)